PCE Inflation, Core PCE Inflation, and U-3 on one graph. Copied the example file and modified.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

In [2]:
from fredapi import Fred

API_KEY_PATH = "fred_api_key.txt" 
fred = Fred(api_key_file = API_KEY_PATH)

In [ ]:
import graph_templates

pio.templates.default = 'fed_2025'

# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "dual_mandate"

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = "../graph_output"


4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Core PCE": "PCEPILFE",
    "Headline PCE": "PCEPI",
    "Unemployment Rate": "UNRATE"
}

In [5]:
today = dt.date.today()

core_sticky_srs = fred.get_series(
    series_id=series_codes["Unemployment Rate"],
    observation_start=dt.date(2000, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Unemployment Rate")

core_srs = fred.get_series(
    series_id=series_codes["Core PCE"],
    observation_start=dt.date(2000, 1, 1),
    observation_end=today,
    units='pc1'
).rename("Core PCE")

headline_srs = fred.get_series(
    series_id=series_codes["Headline PCE"],
    observation_start=dt.date(2000, 1, 1),
    observation_end=today,
    units='pc1'
).rename("Headline PCE")

joined_df = pd.concat(
    [core_sticky_srs, core_srs, headline_srs], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,Unemployment Rate,Core PCE,Headline PCE
Date,,,
2025-02-01,4.1,2.94932,2.67605
2025-03-01,4.2,2.69969,2.34233
2025-04-01,4.2,2.62240,2.23640
2025-05-01,4.2,2.75771,2.41593
2025-06-01,4.1,2.79324,2.58250


In [6]:
# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Inflationa and Unemployment<br><sup>Monthly, Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    yaxis_title="Change (YoY)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH + f"/{GRAPH_NAME}.html")